In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
import os
import time

from datetime import datetime
import pandas as pd
import openpyxl


try:
    from login_info import ID, PW
except:
    ID = input("아이디: ")
    PW = input("비번: ")


target_key = input("연관검색어를 확인할 키워드 :  ")
print(os.getcwd())
address = input("""저장할 or 위의 디렉토리 주소값 복사 + 붙여넣기: """)
address_ = "/" + address
print(f"파일 생성 될 위치 : {address_}")




In [ ]:
driver.close()

In [46]:
address_dict = {'download.default_directory': address}
op = Options()
# op.add_experimental_option('prefs', {'download.default_directory': r'/Users/hannahlee/Desktop/data_science'})
op.add_experimental_option('prefs', address_dict)
driver = webdriver.Chrome(options=op)
driver.get("https://searchad.naver.com/")
pop_up = driver.find_element(By.XPATH,'//*[@id="container"]/main/div[2]/div/button')
pop_up.click()
time.sleep(2)
naver_login = driver.find_element(By.XPATH,'//*[@id="container"]/main/div/div[1]/home-login/div/div/fieldset/button')
EC.element_to_be_clickable((By.XPATH,'//*[@id="container"]/main/div/div[1]/home-login/div/div/fieldset/button'))
naver_login.click()
time.sleep(2)
# ID, 비번
script = f"""
document.getElementById('id').value = '{ID}'
document.getElementById('pw').value = '{PW}'
"""
time.sleep(2)
driver.switch_to.window(driver.window_handles[-1])
driver.execute_script(script)
time.sleep(2)

In [24]:
naver_login = driver.find_element(By.CLASS_NAME,'naver_login_btn')
naver_login.click()
time.sleep(2)
driver.switch_to.window(driver.window_handles[-1])

naver_login = driver.find_element(By.XPATH, '//*[@id="container"]/main/div/div[1]/home-login/div/div/button')
naver_login.click()
time.sleep(2)
driver.switch_to.window(driver.window_handles[-1])
time.sleep(2)

In [42]:
driver.switch_to.window(driver.window_handles[0])
# driver.close()

In [ ]:


# ID, 비번
script = f"""
document.getElementById('id').value = '{ID}'
document.getElementById('pw').value = '{PW}'
"""
driver.execute_script(script)

login_btn = driver.find_element(By.ID, "log.login")
login_btn.click()
time.sleep(2)
driver.switch_to.window(driver.window_handles[-1])
time.sleep(1)
driver.close()
time.sleep(1)
driver.switch_to.window(driver.window_handles[-1])
time.sleep(1)
welcome_btn = driver.find_element(By.XPATH, '//*[@id="mat-dialog-0"]/naver-login-confirm/div[2]/div/div[3]/button')
welcome_btn.click()
try:
    guide_btn = driver.find_element(By.XPATH, '/html/body/marvel-root/wrap/welcome-beginner-layer-popup/div[2]/div[1]/a')
    guide_btn.click()
except:
    pass
keyword_tool = driver.find_element(By.XPATH, '//*[@id="container"]/my-screen/div/div[1]/div/my-screen-board/div/div[1]/ul/li[3]/a')
keyword_tool.click()
driver.switch_to.window(driver.window_handles[-1])
time.sleep(3)
while True:
    try:
        keyword_search = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div/textarea')
        keyword_search.clear()
        keyword_search.send_keys(target_key)
        break
    except:
        time.sleep(2)
search_btn = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div/div/div[2]/div[1]/div[1]/div[3]/button')
search_btn.click()
time.sleep(2)
download_btn = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div/div/div[2]/div[1]/div[2]/div[1]/div/button')
download_btn.click()
time.sleep(5)


# 본인 directory e.g.'/Users/...'
directory = address
recent = [None, 0]
for file in os.listdir(directory):
    file_path = directory + '/' + file
    tmp = os.path.getctime(file_path)
    if tmp > recent[1]:
        if file_path[-4:] == "xlsx":
            recent = [file_path, tmp]
print(f"사용할 파일: {recent[0]}")
        

    

load_xlsx = openpyxl.load_workbook(recent[0])
load_sheet = load_xlsx['sheet']

df = pd.DataFrame(load_sheet.values)
header = df.iloc[0]
df = df[1:]
df.rename(columns=header, inplace=True)
df.set_index('연관키워드', inplace=True)
df["월간검색수(PC)"] = df["월간검색수(PC)"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
df["월간검색수(모바일)"] = df["월간검색수(모바일)"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
df = df.astype({"월간검색수(PC)" : 'int', "월간검색수(모바일)": 'int'})
df["월간검색수(총)"] = df['월간검색수(PC)'] + df['월간검색수(모바일)']
df = pd.DataFrame(df["월간검색수(총)"])

df["총상품수"] = None
# df["총상품수"] = 1



shopping = "https://shopping.naver.com/home"
driver.get(shopping)
search_btn = driver.find_element(By.CLASS_NAME, '_searchInput_button_search_h79Dk')
search_btn.click()

keyword_num = df.count()[0]

for i in range(keyword_num):
    try:
        tmp = df.iloc[i]
        search_input = driver.find_element(By.CSS_SELECTOR, '.searchInput_search_input__vLBeq')
        search_input.clear()
        search_input.send_keys(tmp.name)
        search_btn = driver.find_element(By.CLASS_NAME, 'searchInput_btn_search__GH316')
        search_btn.click()
        total_items = driver.find_element(By.CSS_SELECTOR, '.subFilter_num__S9sle').text
        tmp[1] = int(total_items.replace(",", ""))
        df.iloc[i] = tmp
        if i % 5 == 0:
#             time.sleep(3)
            time.sleep(2)
        if i % 10 == 0:
#             time.sleep(5)
            time.sleep(3)
#         if i % 250 == 0:
#             time.sleep(10)
        if i == 15:     ## 네이버 차단 방지용 (임시)
            break
            
    except:
        continue


tmp = df["총상품수"]
tmp = tmp.dropna().astype("float")
to_fill = tmp.mean() + tmp.median() / 2
to_fill
df["총상품수"] = df["총상품수"].fillna(to_fill)

df["경쟁률"] = df["월간검색수(총)"] / df["총상품수"]




df = df[df["월간검색수(총)"] > 1000]  # 조건 1

df["조건부합"] = 0
df.loc[df["월간검색수(총)"] > df["총상품수"], "조건부합"] = 1 #조건 2
df[df["조건부합"] != 0] # 조건 2 필수일 때



# 조건 3 & 4

import time

# for i in range(df.count()[0]):
for i in range(15): # 일부만 테스트
    satisfied = 0
    name = df.iloc[i].name
    
    search_input = driver.find_element(By.CLASS_NAME, 'searchInput_search_input__vLBeq')
    search_input.clear()
    search_input.send_keys(name)
    search_btn = driver.find_element(By.CLASS_NAME, 'searchInput_btn_search__GH316')
    search_btn.click()
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(1)

    items = driver.find_elements(By.CSS_SELECTOR, '.basicList_info_area__TWvzp')
    cnt = 1
    divider = 1
    total_review = 0

    for item in items:
        review = None
        infos = item.find_elements(By.CSS_SELECTOR, '.basicList_etc__LSkN_:not(.basicList_star__UzKiv)')
        for info in infos:
            if info.text[:2] == "리뷰":
                if info.text[:4] == "리뷰별점":
                    review = int(info.find_element(By.CSS_SELECTOR, '.basicList_num__sfz3h').text.replace(',', ''))
                else:
                    review = int(info.text[2:].replace(",", ''))
        if review != None:
            total_review += int(review)
            divider += 1
        if cnt == 5:
            review_average = total_review / divider
            if review_average > 100:
                satisfied += 1
            total_review = 0
            divider = 1
        elif cnt == 10:
            review_average = total_review / divider
            if review_average < 100: ## 조건 다시 체크..? 리뷰가 하나도 없어도 + 1pt?
                satisfied += 1
        cnt += 1
    
        if cnt == 11:
            break
                
    df["조건부합"][name] += satisfied
    if i%5 == 0:
        time.sleep(1)
driver.close()   
    
result_df = df[df["조건부합"] >= 2]
result_df = result_df.sort_values(by=["경쟁률"], ascending=[True]) 

time_stamp = str(datetime.now()).split(" ")[0].replace("-", "")
result_df.to_excel(f"황금키워드_{target_key}_{time_stamp}.xlsx")


result_li = []
for i in range(result_df.count()[0]):
    result_li.append(result_df.iloc[i].name)
print(result_li)